# 1. 엑셀 파일 불러오기

In [1]:
import pandas as pd
import numpy as np

In [39]:
df=pd.read_excel("sample.xlsx")
df

,body,neighbor,likes,isAd
0,결혼 전 신혼 살림 빠짐없이 차리면서\n\n이만~하면 되었다~~했고요\n\n그 이후...,10,5,1
1,"< 장점 >\n\n \n\n1. 리모컨이 있어서 편함 \n\n2. 상하, 좌우, 다...",2,6,0


In [3]:
y_data=np.array(df.isAd.tolist()) #종속변수 isAd

# 2. 의미없는 부분 제거
1. 공백 (\u200b)
2. \n
3. 문장부호

In [4]:
import re

In [5]:
#전처리 함수 정의: 문장을 IN
def cleaning(x):
    x=re.sub('\W', ' ', x) #특수문자 제거
    x=re.sub('([ㄱ-ㅎㅏ-ㅣ]+)', '', x) #한글 자음 모음 제거
    return x

#전체 행에 전처리 적용
df['body']=df.body.apply(cleaning)

# 2. 형태소 분석기 이용해 토큰화
형태소 분석기는 [Komoran](https://komorandocs.readthedocs.io/ko/latest/pykomoran/tutorial.html) 사용 <- 메소드 정리 페이지 

nouns : 명사 추출  
morphs : 형태소 추출  
pos : 품사 부착  

In [6]:
from konlpy.tag import *

### 사용자 사전, 분석대상 품사 리스트 추가  
1. userdict.txt 고유명사, 신조어 등을 새로 등록. 이들 단어에 품사 태그가 제대로 붙게 한다  
2. pos_table.txt 분석대상 품사를 등록. 불필요한 품사를 제외한다 [komoran 품사태그](https://docs.komoran.kr/firststep/postypes.html)  

In [7]:
komoran=Komoran(userdic='userdict.txt')
pos=pd.read_csv('pos_table.txt', header=None, names='p')
poslist=pos.p.tolist()

[원하는 품사만 추출](https://wikidocs.net/33799)  
poslist에 있는 분석대상 품사만 추출한다  

In [8]:
# 전체 행에 품사 부착 -> 각 행은 ('word', '품사') 튜플의 리스트
df['body']=df.body.apply(komoran.pos)

In [9]:
# 품사 추출 함수 정의; 품사가 부착된 리스트를 IN
def PosPicker(sentence):
    clean_words=[]
    for word in sentence:
        if word[1] in poslist: # 품사가 분석대상 리스트에 있으면
            clean_words.append(word[0]) # 해당 단어를 clean_words에 추가한다
    x=' '.join(clean_words)
    return x

In [10]:
# 전체 행에 품사추출함수 적용
df['body']=df.body.apply(PosPicker)

In [11]:
# 전체 행을 다시 형태소 리스트로 변환
df['body']=df.body.apply(komoran.morphs)

# 3. 불용어 제거
불용어 리스트 stopwords.xlsx에서 관리. 2단계에서 제거되지 않은 불용어를 제거한다   

In [12]:
import numpy as np
stopwords=pd.read_excel("stopwords.xlsx")
stoplist=np.array(stopwords.words.tolist())

In [13]:
# 불용어 제거 함수 정의: 형태소 기준으로 나뉜 리스트를 IN 
def DelStops(sentence): 
    result=[]
    for word in sentence:
        if word not in stoplist:
            result.append(word)
    return result

In [14]:
# 전체 행에 불용어 제거 함수 적용
df['body']=df.body.apply(DelStops)

# 4. 정수 인코딩
정수 인코딩의 원리 https://wikidocs.net/34264   

**TfidVectorizer** [https://wikidocs.net/34265] 더 자세한 설명 [https://wikidocs.net/31698]  
**가중치 부여 문서 벡터화**   
: TF-IDF (Term Frequency - Inverse Document Frequency, 단어 빈도-역문서 빈도)
단어 꾸러미 (BOW(Bag of Words)) 인코딩 벡터에서
여러 문서에서 사용된 단어의 특성에 대해 문서 구별 능력이 떨어진다고 보아 낮은 가중치를 곱해 중요도를 떨어 뜨리기

In [15]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
#벡터화 순서는 크게 3단계

count=CountVectorizer()
tfid=TfidfVectorizer()

# count.fit_transform(result)
# tfid.fit_transform(result)

# feature_matrix=count.transform(result)
# feature_matrix2=tfid.transform(result)

In [17]:
# 단어 리스트를 한 문장으로 합치는 함수 정의 : 단어 리스트 IN
def wordjoin(x):
    return ' '.join(x)

In [18]:
# 전체 행에 대해 문장화 적용
df.body=df.body.apply(wordjoin)

In [32]:
print(tfid.get_feature_names())
print(tfid.vocabulary_)

['ㄴ지', '가능', '가전', '건조기', '결혼', '고개', '공기', '그런', '그리', '기업', '노예', '단계', '단점', '때문', '라고', '리모컨', '마케팅', '많이', '머리', '먼저', '모드', '무시', '바닥', '바람', '발맞추', '부족', '분명', '비싸', '빠짐없이', '사이', '살림', '상하', '생기', '생소', '선풍기', '세상', '소음', '숙이', '순환', '스마트', '스타일', '시대', '시원', '신혼', '애쓰', '야금', '어쩌면', '에어', '에어컨', '에어프라이어', '우리집', '유아', '이랑', '이름', '이제', '이후', '익숙', '인테리어', '자꾸', '장점', '정도', '제가', '조용', '조절', '좌우', '지금', '지나', '차리', '처음', '타이머', '터보', '포장', '필수', '필요', '해보면', '혼수', '효과']
{'결혼': 4, '신혼': 43, '살림': 30, '빠짐없이': 28, '차리': 67, '이후': 55, '야금': 45, '부족': 25, '자꾸': 58, '때문': 13, '필요': 73, '가전': 2, '생기': 32, '기업': 9, '머리': 18, 'ㄴ지': 0, '분명': 26, '혼수': 75, '세상': 35, '많이': 17, '에어프라이어': 49, '스타일': 40, '애쓰': 44, '무시': 21, '비싸': 27, '건조기': 3, '필수': 72, '지금': 65, '그런': 7, '에어': 47, '선풍기': 34, '에어컨': 48, '시대': 41, '이제': 54, '라고': 14, '처음': 68, '이름': 53, '생소': 33, '지나': 66, '먼저': 19, '그리': 8, '사이': 29, '익숙': 56, '우리집': 50, '어쩌면': 46, '마케팅': 16, '노예': 10, '포장': 71, '해보면': 74, '발맞추': 24, '스마트': 3

클렌징이 끝난 body 열을 넣으면 여러 문서들에 나타나는 단어들에 대해서는 가중치(TF-IDF)를 낮추고, 각 문서를 구별하는 중요 단어에는 가중치를 올려 matrix로 리턴받을 수 있다. 

In [35]:
# 문장 벡터화 (TFID)
tfid.fit_transform(df.body)

#bodylist=df.body.tolist()  <- 하나의 리스트로 변환하지 않아도 됨
#tfid.fit_transform(bodylist)

<2x77 sparse matrix of type '<class 'numpy.float64'>'
	with 79 stored elements in Compressed Sparse Row format>

In [20]:
# matrix로 저장 (TFID) -> 문장 수 * 단어 수 크기의 matrix가 생성된다
feature_matrix2=tfid.transform(df.body)
print(feature_matrix2)
# feature_matrix3=tfid.transform(bodylist)
# print(feature_matrix3)

  (0, 75)	0.1054019430750602
  (0, 74)	0.1054019430750602
  (0, 73)	0.2108038861501204
  (0, 72)	0.31620582922518065
  (0, 71)	0.1054019430750602
  (0, 68)	0.1054019430750602
  (0, 67)	0.1054019430750602
  (0, 66)	0.1054019430750602
  (0, 65)	0.07499435060108835
  (0, 58)	0.1054019430750602
  (0, 56)	0.1054019430750602
  (0, 55)	0.1054019430750602
  (0, 54)	0.1054019430750602
  (0, 53)	0.2108038861501204
  (0, 50)	0.1054019430750602
  (0, 49)	0.1054019430750602
  (0, 48)	0.07499435060108835
  (0, 47)	0.2108038861501204
  (0, 46)	0.1054019430750602
  (0, 45)	0.2108038861501204
  (0, 44)	0.1054019430750602
  (0, 43)	0.1054019430750602
  (0, 41)	0.2108038861501204
  (0, 40)	0.1054019430750602
  (0, 39)	0.1054019430750602
  :	:
  (1, 65)	0.09592886111521996
  (1, 64)	0.1348246671579957
  (1, 63)	0.2696493343159914
  (1, 62)	0.1348246671579957
  (1, 61)	0.1348246671579957
  (1, 60)	0.1348246671579957
  (1, 59)	0.1348246671579957
  (1, 57)	0.1348246671579957
  (1, 52)	0.1348246671579957
  (1

**matrix에 열 추가**  
body는 2*77의 matrix로 변환되었다. 그 오른쪽에 neighbor, likes 등의 열을 추가한다

In [44]:
new_matrix=np.zeros((feature_matrix2.shape[0], feature_matrix2.shape[1]+2))

new_matrix[:,:-2]=feature_matrix2.toarray()
new_matrix[:,-2]=df.neighbor
new_matrix[:,-1]=df.likes

In [45]:
new_matrix

array([[ 0.10540194,  0.        ,  0.31620583,  0.10540194,  0.21080389,
         0.        ,  0.        ,  0.10540194,  0.10540194,  0.10540194,
         0.10540194,  0.        ,  0.        ,  0.10540194,  0.10540194,
         0.        ,  0.10540194,  0.10540194,  0.10540194,  0.10540194,
         0.        ,  0.10540194,  0.        ,  0.        ,  0.10540194,
         0.10540194,  0.21080389,  0.10540194,  0.10540194,  0.10540194,
         0.10540194,  0.        ,  0.21080389,  0.10540194,  0.10540194,
         0.10540194,  0.        ,  0.        ,  0.        ,  0.10540194,
         0.10540194,  0.21080389,  0.        ,  0.10540194,  0.10540194,
         0.21080389,  0.10540194,  0.21080389,  0.07499435,  0.10540194,
         0.10540194,  0.        ,  0.        ,  0.21080389,  0.10540194,
         0.10540194,  0.10540194,  0.        ,  0.10540194,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.07499435,  0.10540194,  0.10540194,  0.1

# 5. 모델링
(나이브 베이즈 모델)[https://wikidocs.net/35435]  
(랜덤포레스트)  
(딥러닝)  

### 데이터셋 쪼개기

In [22]:
## 1) 종속변수와 독립변수로 나누기 
# from sklearn.model_selection import train_test_split

# train_data=df.drop(['isAd'],axis=1)
# target_data=df.isAd

# ## 2) 각각을 8:2로 나누기 (train과 test 셋)
# x_train,x_test,y_train,y_test=train_test_split(train_data, target_data, test_size=0.2)

# ## 3) train을 다시 8:2로 나누기 (train과 valid)
# x_train,x_valid,y_train,y_valid=train_test_split(x_train,y_train, test_size=0.2)


### 랜덤포레스트

In [47]:
from sklearn.ensemble import RandomForestClassifier

forest= RandomForestClassifier(n_estimators=10) 
forest.fit(new_matrix, target_data)

print(" training set : ", forest.score(new_matrix, target_data))
# print(" validation set : ", forest.score(feature_matrix2, target_data))

 training set :  1.0


분류에 중요한 특성은?

최적의 max_depth, n_estimators 찾기

### 나이브 베이즈 모델
https://wikidocs.net/34389

In [28]:
import nltk
from sklearn.naive_bayes import BernoulliNB
from sklearn.externals import joblib
#pip install joblib

In [ ]:
# joblib.dump(tfid, 'model/test_1.pkl')

In [48]:
#모델 생성
model=BernoulliNB(alpha=1.0)
model.fit(new_matrix, target_data) #정수화한 matrix를 넣어야 함

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [49]:
print(model.score(new_matrix, target_data))

1.0


### 딥러닝

# 6. 성능 평가

# 7. 모델 예측
학습에 사용하지 않은 값 넣어 예측해보기

# 빈도 시각화

In [ ]:
# 그래프 한국어 설정
import matplotlib as mpl
import matplotlib.pylab as plt
plt.rcParams["font.family"]="Malgun Gothic"

In [ ]:
from nltk import Text

In [ ]:
kolaw=Text(komoran.nouns(temp))
kolaw.plot(10)
plt.show()